In [1]:
# для примера использую библиотеку neuralprophet
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
import os
import pickle

In [12]:
airports = pd.read_csv('airports.csv')
fligshts = pd.read_csv('flights.csv')

WARNING - (py.warnings._showwarnmsg) - /var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_93379/4155919845.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  fligshts = pd.read_csv('flights.csv')



In [13]:
#объединяю данные, что бы в данных было найменование аэропорта и код аэропорта
#датасет с цифровой расшифровкой аэропортов не нашел, здесь так же удаляю эти данные

fligshts = fligshts.merge(airports, how='right', left_on='ORIGIN_AIRPORT', right_on='IATA_CODE')

In [14]:
#добавляю колонку с датой в формате datetime64

fligshts['Date'] = fligshts[['YEAR','MONTH','DAY']].astype('str').agg('-'. join , axis= 1)
fligshts['Date'] = pd.to_datetime(fligshts['Date']).dt.date.astype('datetime64[ns]')

#удаляю колонки с датой, месяцем, годом

fligshts.drop(['YEAR', 'DAY', 'MONTH'], axis=1, inplace=True)


In [15]:
fligshts.head(2)

,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,Date
0,4,EV,5103,N846AS,ABE,DTW,600,552.0,-8.0,12.0,...,NaN,NaN,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.4404,2015-01-01
1,4,EV,5579,N136EV,ABE,ATL,635,628.0,-7.0,13.0,...,NaN,NaN,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.4404,2015-01-01


In [16]:
fligshts = fligshts[['Date', 'ORIGIN_AIRPORT', 'DEPARTURE_DELAY', 'DESTINATION_AIRPORT', 'ARRIVAL_DELAY']].reset_index()
fligshts.dropna(inplace=True)

In [17]:
fligshts['ARRIVAL_DELAY'] = fligshts['ARRIVAL_DELAY'].astype('int')
fligshts['DEPARTURE_DELAY'] = fligshts['DEPARTURE_DELAY'].astype('int')


In [18]:
#здесь формирую колонку PROBABILITY - если самолет опоздал, то значение 0
# из опоздания прилета вычитаю задержку на вылете, что бы исключить вину в этой задержке принимающего аэропорта
#прилетел раньше или в срок - значение 1


fligshts['DELAY'] = fligshts['ARRIVAL_DELAY'] - fligshts['DEPARTURE_DELAY']
fligshts['PROBABILITY'] = np.where(fligshts['DELAY']>0, 0, 1)


In [19]:
fligshts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5231130 entries, 0 to 5332913
Data columns (total 8 columns):
 #   Column               Dtype         
---  ------               -----         
 0   index                int64         
 1   Date                 datetime64[ns]
 2   ORIGIN_AIRPORT       object        
 3   DEPARTURE_DELAY      int64         
 4   DESTINATION_AIRPORT  object        
 5   ARRIVAL_DELAY        int64         
 6   DELAY                int64         
 7   PROBABILITY          int64         
dtypes: datetime64[ns](1), int64(5), object(2)
memory usage: 359.2+ MB


In [ ]:
# получаю список аэропортов вылета

ORIGIN_AIRPORT = list(fligshts.ORIGIN_AIRPORT.unique())
ORIGIN_AIRPORT

In [ ]:
# создаем каталоги
for airport in ORIGIN_AIRPORT:
    os.mkdir("{}".format(airport))

In [ ]:
# и далее идея следующая - для каждого аэропорта вылета в цикле перебрать аэропорты прилета

for airport in ORIGIN_AIRPORT:
    df = fligshts.query(f'ORIGIN_AIRPORT == "{airport}"') #датафрейм с конкретным аэропортом прилета
    DESTINATION_AIRPORT = list(df.DESTINATION_AIRPORT.unique()) #список аэропортов прилета для конкретного аэропорта вылета
    for d_airport in DESTINATION_AIRPORT:
        A = df.query(f'DESTINATION_AIRPORT == "{d_airport}"') #фрейм с конкретным аэропортом прилета
        B = A.groupby('Date')['PROBABILITY'].mean().reset_index() #фрейм с датой и средним значением вероятности задержки в этот аэропорт по дням
        B.columns = ['ds', 'y'] #привожу колонки к наименованию, которые необходимы для модели
        m = NeuralProphet() #создаем экземпляр класса
        metrics = m.fit(B, freq="D") #обучаем модель
        with open('{}'.format(airport)+'/'+'{}'.format(d_airport)+'.pkl', 'wb') as f:
            pickle.dump(metrics, f)



#хочу оформить все в streamlit - нашел, как сделать пользовательский ввод, например пользователь вводит город вылета,
#далее формируется список аэропортов в этом городе, выбирается аэропотр, например, дата вылета

# выгружаю модели из соответствующей папки - в зависимости от выбранного аэропорта
# делаю предсказание, отвечаю на вопрос задания и сопутствующую визуализацию строю    

In [ ]:
pred = m.make_future_dataframe(B, periods=2)
forecast = m.predict(pred)